## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
revised_application_df = application_df.drop(columns=['EIN','NAME'])

In [3]:
# Generate our categorical variable lists
categories = revised_application_df.dtypes[revised_application_df.dtypes == "object"].index.tolist()
categories 

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_types = revised_application_df['APPLICATION_TYPE'].value_counts()

In [5]:
# Filter out application types to be kept and create a list of the application types to be removed
app_types_df = pd.DataFrame(app_types)
result_df = app_types_df[app_types_df['count'] < 1000]
result_df = result_df.reset_index().drop(columns='count')
remove_list = result_df['APPLICATION_TYPE'].to_list()
remove_list

['T8',
 'T7',
 'T10',
 'T9',
 'T13',
 'T12',
 'T2',
 'T25',
 'T14',
 'T29',
 'T15',
 'T17']

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = remove_list

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"App_Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3           27037
App_Other     2266
T4            1542
T6            1216
T5            1173
T19           1065
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_types = revised_application_df['CLASSIFICATION'].value_counts()
classification_types

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# Filter out classification types to be kept and create a list of the classification types to be removed
classification_types_df = pd.DataFrame(classification_types)
class_result_df = classification_types_df[classification_types_df['count'] < 1000]
class_result_df = class_result_df.reset_index().drop(columns='count')
class_remove_list = class_result_df['CLASSIFICATION'].to_list()

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_remove_list
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Class_Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000          17326
C2000           6074
C1200           4837
Class_Other     2261
C3000           1918
C2100           1883
Name: count, dtype: int64

In [10]:
# Convert categorical and application data to numeric with `pd.get_dummies`
class_data = pd.get_dummies(application_df['CLASSIFICATION'])
app_data = pd.get_dummies(application_df['APPLICATION_TYPE'])
affiliation_data = pd.get_dummies(application_df['AFFILIATION'])
use_case_data = pd.get_dummies(application_df['USE_CASE'])
organization_data = pd.get_dummies(application_df['ORGANIZATION'])
income_amount_data = pd.get_dummies(application_df['INCOME_AMT'])
dummy_application_df = pd.concat([class_data,app_data,application_df,affiliation_data,use_case_data,organization_data,income_amount_data],axis=1)
dummy_application_df

,C1000,C1200,C2000,C2100,C3000,Class_Other,App_Other,T19,T3,T4,...,Trust,0,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M
0,True,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,True,False,...,True,False,False,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,True,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
34295,False,False,False,False,True,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
34296,False,False,True,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
34297,False,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [11]:
dummy_application_df.columns

Index(['C1000', 'C1200', 'C2000', 'C2100', 'C3000', 'Class_Other', 'App_Other',
       'T19', 'T3', 'T4', 'T5', 'T6', 'EIN', 'NAME', 'APPLICATION_TYPE',
       'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS',
       'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL',
       'CompanySponsored', 'Family/Parent', 'Independent', 'National', 'Other',
       'Regional', 'CommunityServ', 'Heathcare', 'Other', 'Preservation',
       'ProductDev', 'Association', 'Co-operative', 'Corporation', 'Trust',
       '0', '1-9999', '10000-24999', '100000-499999', '10M-50M', '1M-5M',
       '25000-99999', '50M+', '5M-10M'],
      dtype='object')

In [12]:
dummpy_application_df = dummy_application_df.drop(columns=['EIN','NAME','APPLICATION_TYPE','STATUS','ASK_AMT','AFFILIATION',
                                                          'CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT',
                                                           'SPECIAL_CONSIDERATIONS','ASK_AMT'])

In [13]:
# Split our preprocessed data into our features and target arrays
y = dummpy_application_df["IS_SUCCESSFUL"].values
X = dummpy_application_df.drop(columns = "IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y) 

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 1
hidden_nodes_layer1 = 1
hidden_nodes_layer2 = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 1)                   │               2 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │               2 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │               2 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6 (24.00 B)

 Trainable params: 6 (24.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [27]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100) 

Epoch 1/100


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_15" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (None, 36)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 36), dtype=float32)
  • training=True
  • mask=None

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 25724, but received input with shape (None, 36)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 36), dtype=float32)
  • training=False
  • mask=None

In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE